# クイックスタート1

このノートのクイックスタートは、2025年2月時点での Microsoft の Azure AI Foundry の[ドキュメント](https://learn.microsoft.com/ja-jp/azure/ai-studio/quickstarts/get-started-code)に記載されている内容になります。

※仕様やドキュメントの変更に伴い、この内容が古くなる可能性があります。


## 前提条件


このノートの内容を学習するにあたって、以下が必要になります。

- Visial Studio Code または このノートブックを実行可能な Jupyter が動作する環境
- Azure サブスクリプション
- 作成済みの Azure AI Foundry プロジェクトとデプロイしたモデル
- Python 3.8 以降
- Python に関する基礎知識
- 必要な Python パッケージ（下記のコードでインストールする）


In [ ]:
# 必要な Python パッケージのインストール
%pip install python-dotenv azure-identity azure-ai-projects azure-ai-inference

## Azure AI Foundry で シンプルな AI 推論チャットアプリを構築する


このノートでは、Azure AI Foundry SDK を使用して、シンプルチャットアプリケーションを開発する方法について説明します。
Python アプリケーションのコードの一部として定義したプロンプトで 推論モデル（LLM）を呼び出して、推論モデル（LLM）からの出力を評価します。


### 準備


このクイック スタートを始める前に、Azure AI Foundry ポータルから、ハブ リソースとプロジェクト リソースを作成して `gpt-4o-mini` モデルをプロジェクトへデプロイしてください。

> [Azure AI Foundry プレイグラウンドのクイックスタート](https://learn.microsoft.com/ja-jp/azure/ai-studio/quickstarts/get-started-playground) を完了している場合、必要な作業は完了しています。

Visual Studio Code でこのノートを開き、[ドキュメントのこのセクション](https://code.visualstudio.com/docs/python/environments#_creating-environments) を参考にしながら、ワークスペースに Python 仮想環境を作成します。仮想環境を作成する際には、このフォルダにある `requirements.txt` を選択すると、このノートの Python アプリケーションコードの実行に必要なライブラリを Python 仮想環境にインストールすることができます。

仮想環境の準備ができたら、このノートにある Python コードをステップごとに実行します。


### チャットアプリのコードサンプル

#### 1. Azure AI Project の Python ライブラリを読み込みます

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

#### 2. 環境変数ファイル (`.env`) から 設定情報を読み込みます


Azure AI Foundry プロジェクトの接続文字列と推論モデルの名前の文字列を `.env` ファイルから読み込みます。

> `.env` ファイルがない場合は `_env` ファイルをテンプレートとして用意していますので、このファイルをコピーして `.env` ファイルを作成してください。

`.env` ファイルには、以下の "**キー** = **値**" のペアを記述します。

- プロジェクトの接続文字列：PROJECT_CONNECTION_STRING="**<AI Foundry プロジェクトの接続文字列>**"
- 推論モデルの名前：MODEL_NAME="**<デプロイしたLLMモデル名>**"


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('.env', override=True)
project_connection_string = os.getenv('PROJECT_CONNECTION_STRING')
model_name_string = os.getenv('MODEL_NAME')

#### 3. Azure AI Foundry プロジェクトに接続します (プロジェクト クライアントを作成します)


`AIProjectClient` のクラスファクトリメソッド `from_connection_string` を使って Azure AI Foundry プロジェクトのクライアントを作成して、プロジェクトに接続します。
`from_connection_string` では、(`.env` ファイルに記載した) 環境変数 `PROJECT_CONNECTION_STRING` で指定したプロジェクト接続名を使用します。

Azure AI Foundry プロジェクトへの接続のためには、Microsoft Entra ID のユーザーと Azure CLI (`az login`) を使って、事前に Azure へのログイン認証を行っておく必要があります。

`AIProjectClient` の同期クライアントを作成するには、次の手順を実行します：


In [ ]:
try:
    credential = DefaultAzureCredential()
    print(f"✅ Azure 資格情報の初期化が成功しました.")
except Exception as e:
    print(f"❌ Azure 資格情報の初期化が以下の理由で失敗しました: {str(e)}")

try:
    project = AIProjectClient.from_connection_string(
        conn_str=project_connection_string,
        credential=credential
    )
    print(f"✅ プロジェクトクライアントの初期化が成功しました.")
except Exception as e:
    print(f"❌ プロジェクトクライアントの初期化が以下の理由で失敗しました: {str(e)}")

#### 4. チャット補完クライアントを作成します


プロジェクト クライアントを使って、プロジェクト内の推論モデルの AI サービスに接続します。

この例では、`get_chat_completions_client()` を使って、チャット補完クライアント `chat` (`ChatCompletionsClient` クラスのインスタンス) を作成します。


In [ ]:
try:
    chat = project.inference.get_chat_completions_client()
    print(f"✅ チャットクライアントの初期化が成功しました.")
except Exception as e:
    print(f"❌ チャットクライアントの初期化が以下の理由で失敗しました: {str(e)}")

#### 5. チャット補完クライアントを使って問い合わせを行います


システムプロンプトと問い合わせメッセージをセットして、
作成したチャット補完クライアント (`chat`) を使って、推論モデルからの回答を取得 (`complete()`) します。

- システムプロンプトと問い合わせメッセージを組み合わせてメッセージとします
- 指定した推論モデルに問い合わせを行います

システムプロンプトを変更して、推論モデルからの回答を評価してください。


In [ ]:
system_prompt_message = "あなたは 2350 年のテクノ パンク ロッカーのように話す AI アシスタントです。クールだけどクール過ぎない。わかりましたか??"
user_query_message = "税金の手続きを手伝ってくれませんか？私はフリーランサーです。"

In [ ]:
system_prompt_message = "あなたは税務署の職員で、税金に詳しいプロフェッショナルです。質問に対して手続きを細かく説明してください。わかりましたか??"
user_query_message = "税金の手続きを手伝ってくれませんか？私はフリーランサーです。"

In [ ]:
from azure.ai.inference.models import SystemMessage, UserMessage

response = chat.complete(
    model=model_name_string,
    messages=[
        SystemMessage(
            content=system_prompt_message,
        ),
        UserMessage(
            content=user_query_message,
        ),
    ],
)
print(response.choices[0].message.content)